This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# Convolutional Neural Networks 與影像分類

請記得先複製一份在你自己的 Google 帳號底下：
`檔案` -> `在雲端硬碟中儲存副本`

也請記得改變 colab 設定：
1. `工具` -> `設定` -> `編輯器` -> 將 `縮排寬度` 改為 `4`
2. `編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`

## 展示題：紅貴賓？炸雞？

根據某專業機構的民調，紅貴賓與炸雞是最容易讓人們搞混的東西。為了避免不小心將紅貴賓吃下肚，我們將用 CNN 來訓練一個影像分類器，分辨照片是紅貴賓（`dog`）還是炸雞（`chicken`）！

![](https://user-images.githubusercontent.com/8934290/57471983-68a86400-725a-11e9-8717-7ae5892aa809.png)

（[原始資料集與圖片來源](https://github.com/buchananwp/LabradoodleOrFriedChicken)）

In [ ]:
!pip install numpy matplotlib torch torchvision

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

### Data Preprocessing

我們先將資料集的壓縮檔下載到 colab server 的硬碟，並且解壓縮到資料夾 `dog-or-chicken`

In [ ]:
! wget https://github.com/SuJiaKuan/fgu_ai_course/raw/main/datasets/dog-or-chicken.zip -O dog-or-chicken.zip && unzip -qo dog-or-chicken.zip

`dog-or-chicken` 這個資料夾包含 training data 的子資料夾（`dog-or-chicken/train`）與 testing data 的子資料夾（`dog-or-chicken/test`），這兩個子資料夾下面又分別有兩種 label 的孫資料夾（`dog-or-chicken/XXX/chicken` 以及 `dog-or-chicken/XXX/dog`），每個孫資料有該 label 的圖片檔案。

資料夾的階層關係如下：
```
dog-or-chicken
├── test
│   ├── chicken
│   └── dog
└── train
    ├── chicken
    └── dog
```

In [ ]:
!ls dog-or-chicken/train/dog

接下來示範如何用 PyTorch 讀取我們自己的資料集。

整體作法跟上次的練習很像，唯一的差別為這次是用 [torchvision.datasets.ImageFolder](https://pytorch.org/vision/stable/datasets.html#imagefolder) 讓程式去讀取硬碟資料夾的圖片，並且根據資料夾名稱來分 label（`chicken` 或 `dog`）。

另外，除了要指定將資料轉成 Tensor 型別之外，因為每張資料集的每張圖片大小不一樣，但 CNN 的 input size 通常是固定的，因此這裡要統一將所有圖片轉成同個 size（244 x 224）。

In [ ]:
import torchvision.transforms as transforms
from torchvision import datasets

# Batch size: how many samples per batch to load.
batch_size = 16

# Image size: input image size to feed into the CNN.
image_size = (224, 224)

# Convert data to torch Tensor.
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
])


# Load the training and testing data from disk.
train_data = datasets.ImageFolder(
    "dog-or-chicken/train",
    transform=transform,
)
test_data = datasets.ImageFolder(
    "dog-or-chicken/test",
    transform=transform,
)

# Create loaders for training and testing data.
train_loader = torch.utils.data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    test_data,
    shuffle=True,
    batch_size=batch_size,
)

我們可以看一下 `DataLoader` 物件的 label 是否如我們預期。另外，也可以抓一把資料來看看狀況。

在這裡可以看到圖片的 shape 為 `[16, 3, 224, 224]`，再複習一次，這種格式稱之為 `NCHW`，分別代表的意義如下：
- `N`：batch size 大小
- `C`：圖片的 channel 數量，灰階圖為 1，彩色圖通常為 3
- `H`：圖片的高度（height）
- `W`：圖片的寬度（width）


In [ ]:
# Show the data labels.
print("Labels: {}".format(train_data.classes))

# Obtain one batch of training images.
images, labels = iter(train_loader).next()

print("Images shape: {}".format(images.shape))
print("Labels shape: {}".format(labels.shape))

### Data Visualization

我們從訓練資料秀幾張圖片來看看！

In [ ]:
# Obtain one batch of training images.
images, labels = iter(train_loader).next()
images = images.numpy()

# Plot the images in the batch, along with the corresponding labels.
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size / 2, idx + 1, xticks=[], yticks=[])
    # To show the image, we need to convert the format from CHW to HWC.
    ax.imshow(images[idx].transpose((1, 2, 0)))
    # Print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(train_data.classes[labels[idx].item()]))

    fig.show()

### Training

開始訓練之前，我們要先定義 CNN 架構。

這裡我們直接使用 [torchvision.models](https://pytorch.org/vision/stable/models.html) 所提供的 ResNet-18，並且將最後一層的 Fully-Connected Layer 從原本 output label 數量 1,000 改成 2。

一般來說，我們會直接使用已經在 ImageNet 下預訓練好的模型，不過為了讓你比較有沒有預訓練的差異，因此這裡先使用沒有預訓練的，後面的練習題會再請你修改成有預訓練的版本。

In [ ]:
from torchvision import models

# Get the pre-trained ResNet-18 model.
model = models.resnet18()
# Replace the last fully-connected layer to our custom layer.
model.fc = torch.nn.Linear(model.fc.in_features, 2)
# Move model from CPU to GPU.
model = model.cuda()

print(model)

我們也要指定要用到的 loss function 以及優化器（更新權重的方法），分別使用 Cross Entropy 以及 Adam。

In [ ]:
# Specify loss function.
criterion = torch.nn.CrossEntropyLoss()

# Specify optimizer.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

接下來正式做訓練的流程，基本上流程與訓練 MLP 幾乎一樣。

In [ ]:
# Number of epochs to train the model.
n_epochs = 10

# Turn on training mode.
model.train()

for epoch in range(n_epochs):
    # Monitor training loss.
    train_loss = 0.0
    
    # Train the model.
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # Move data from CPU to GPU.
        data = data.cuda()
        target = target.cuda()
        # Forward pass:
        # Compute predicted outputs by passing inputs to the model.
        output = model(data)
        # Calculate the loss.
        loss = criterion(output, target)
        # Backward pass:
        # Compute gradient of the loss with respect to model parameters.
        loss.backward()
        # Perform a single optimization step (parameter update).
        optimizer.step()
        # Update running training loss.
        train_loss += loss.item() * data.size(0)
        
    # Calculate average loss over an epoch.
    train_loss = train_loss / len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {}'.format(
        epoch + 1, 
        train_loss
    ))

### Evaluation

訓練完成後，我們可以用模型來做 inference。這裡示範從 testing data 抓一把資料，餵給模型來做預測。要注意的是由於我們定義的模型並沒有包含 Softmax，所以在 inference 的時候我們手動加上去，這樣預測的結果就會是每個 label 的機率，我們就可以將機率做大的 label 作為預測的 label。

In [ ]:
import torch.nn.functional as F

# Turn off training mode (i.e., turn on evaluation mode).
model.eval()

# Obtain one batch of testing images.
images, labels = iter(test_loader).next()

# Get sample outputs.
output = model(images.cuda())
output = F.softmax(output)
print(output.shape)

# Convert output probabilities to predicted class.
_, preds = torch.max(output, 1)

# Prepare images for display.
images = images.numpy()

# Plot the images in the batch, along with predicted and true labels.
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size / 2, idx + 1, xticks=[], yticks=[])
    ax.imshow(images[idx].transpose((1, 2, 0)))

    title = "{} ({})".format(
        preds[idx].item(),
        train_data.classes[labels[idx].item()],
    )
    color = "green" if preds[idx] == labels[idx] else "red"
    ax.set_title(title, color=color)

接下來我們可以用全部的 testing data 來算準確率，分別計算各個 label 的準確率，以及全部之下的準確率。

In [ ]:
class_correct = [0] * 2
class_total = [0] * 2

for data, target in test_loader:
    # Get data outputs.
    data = data.cuda()
    target = target.cuda()
    output = model(data)
    output = F.softmax(output)
    # Convert output probabilities to predicted class.
    _, pred = torch.max(output, 1)
    # Compare predictions to true label.
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(target.shape[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

for label in range(2):
    print("Test Accuracy of {}: {}% ({} / {})".format(
        label,
        100 * class_correct[label] / class_total[label],
        int(np.sum(class_correct[label])),
        int(np.sum(class_total[label]))),
    )
print("Test Accuracy (Overall): {}% ({} / {})".format(
    100 * np.sum(class_correct) / np.sum(class_total),
    int(np.sum(class_correct)),
    int(np.sum(class_total))),
)

## 練習題

目前我用的 ResNet-18，是沒有經過任何預訓練的，因此準確率只能說是差強人意。現在請你將模型改成是有先經過預訓練的版本，並且再重新訓練一次看看，準確率應該至少能提升到 96%。

提示：
1. 請參考 [torchvision.models](https://pytorch.org/vision/stable/models.html) 修改成有預訓練（pretrained）的版本
2. 請修改讀取資料的程式碼區塊，在 `transform` 的 `ToTensor()` 後面再加上一個 [torch.transforms.Normalize](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Normalize)，其中 `mean` 填 `[0.485, 0.456, 0.406]`，
`std` 填 `[0.229, 0.224, 0.225]`（這些數值是從 ImageNet 得來的）。
2. 請記得重新按 `指定 loss function 以及優化器` 這格的程式碼區塊，因為 Adam 是會根據之前訓練的結果來更改一些內容。重新執行這個區塊可以讓 Adam 重新初始化。